## Test with pretrained models

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import os
import time
import cpuinfo
from tqdm import tqdm

### File paths

In [ ]:
train_path = "../data/Train"    # paths for your training and testing dataset
test_path = "../data/Test"
input_parameter = ""            # paths for import and export custom model trainable parameters
output_parameter = ""

### Pretrain Model of use from torchvision

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights
model = resnet50(weights=ResNet50_Weights.DEFAULT)
print(f"Using model {type(model).__name__}")

epochs = 3
batch_size = 64 # adjust to your gpu vram
optimizer = optim.Adam(model.parameters(), lr=0.0003)
loss_function = nn.CrossEntropyLoss()

### Data Preprocess Transformations

In [ ]:
transform_train = transforms.Compose([  # on training dataset
    transforms.RandomRotation(degrees=10),  # introduce varieties for better generalization
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # ImageNet1K stats
    ])

transform_test = transforms.Compose([   # on test dataset
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

### Data Loader

In [ ]:
# Load datasets
train_dataset = datasets.ImageFolder(root=train_path, transform=transform_train)
test_dataset = datasets.ImageFolder(root=test_path, transform=transform_test)

print(f"CPU count: {os.cpu_count()}")
num_workers = max(0, os.cpu_count() // 2)  # Dynamically set num_workers
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

model.fc = nn.Linear(model.fc.in_features, len(train_dataset.classes)) # set output layer to match number of classes
print(f"Classes: {train_dataset.classes}")

### Device of use

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"

if (device == torch.device("cuda")):
    print(f"Using GPU: {torch.cuda.get_device_name(device)}")
else:
    cpu_info = cpuinfo.get_cpu_info()
    print(f"Using CPU: {cpu_info['brand_raw']}")

### Configure model parameters

In [ ]:
# IF NEEDED
# Load custom weight and optimizer states
if os.path.exists(input_parameter):
    checkpoint = torch.load("test_weights.pth", map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
# Selective layer freezing
# change base on your model
# "Early layers are often already well-optimized" by ChatGPT-O4
for name, param in model.named_parameters():
    if "layer4" not in name and "fc" not in name:
        param.requires_grad = False  # Freeze everything except layer4 and fc

In [ ]:
# Move model to device
model.to(device)
print(f"Model is on {next(model.parameters()).device}")

### Training Epochs

In [ ]:
from tqdm import tqdm

for epoch in range(epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    # Training loop with progress bar
    train_bar = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs} [Train]", leave=False)
    for images, labels in train_bar:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = loss_function(outputs, labels)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Calculate statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        # Update progress bar
        train_bar.set_postfix({
            'loss': f"{running_loss / (train_bar.n + 1):.4f}",
            'acc': f"{100. * correct_train / total_train:.2f}%"
        })

    # Calculate training statistics
    train_loss = running_loss / len(train_loader)
    train_acc = 100. * correct_train / total_train

    # Validation phase
    model.eval()
    val_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        val_bar = tqdm(test_loader, desc=f"Epoch {epoch + 1}/{epochs} [Val]", leave=False)
        for images, labels in val_bar:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = loss_function(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

            # Update validation progress bar
            val_bar.set_postfix({
                'loss': f"{val_loss / (val_bar.n + 1):.4f}",
                'acc': f"{100. * correct_val / total_val:.2f}%"
            })

    # Calculate validation statistics
    val_loss /= len(test_loader)
    val_acc = 100. * correct_val / total_val

    # Print epoch summary
    print(f"\nEpoch {epoch + 1}/{epochs} - "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}% | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")

### Training log and data export

In [ ]:
# Code here